In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def data():
    training = pd.read_csv("./training_preprocessed")
    validation = pd.read_csv("./validation_preprocessed")
    test = pd.read_csv("./test_preprocessed")
    X_train = training.drop(columns="y")
    y_train = training["y"]
    X_valid = validation.drop(columns="y")
    y_valid = validation["y"]
    X_test = test
    return X_train, y_train, X_valid, y_valid, X_test

In [3]:
X_train, y_train, X_valid, y_valid, X_test = data()

In [4]:
wandb.init(project="house-price-prediction", entity="luiscostigan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luiscostigan (use `wandb login --relogin` to force relogin)


In [5]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
    
    return model

In [10]:
from sklearn.model_selection import KFold

# Instantiate the validator, have it spit out the indices, generate batches from the indices, and feed those batches to the model.
kf = KFold(n_splits=5)
kf.get_n_splits(X_train, y_train)

5

Currently trying to implement RepeatedKFold cross validation with the Keras Sequential model, while recording model results on WandB.

Sometimes I am getting a failed-1 error from WandB.
    - 

In [13]:
from wandb.keras import WandbCallback

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "random",
  "parameters": {
    "dropout1": {
      "min": 0.0,
      "max": 0.8
    },
    "epochs": {
      "values": [30,50,100]
    },
    "batch_size": {
      "values": [16, 64, 128, 512]
    }
  }
}

config_defaults = {
  "dropout1": 0.5,
  "epochs": 50,
  "batch_size": 64
}

def train():

  mae_per_fold = []
  loss_per_fold = []
  fold_no = 1

  # Go through each split, and get the index number for each (currently 50, 10 x 5 repeated k-fold) 
  for train, test in kf.split(X_train, y_train):

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      # Recreate the model each time for each new batch
      model = None # Not sure if this step is necessary
      model = create_model()

      # Fit model on new batches
      model.fit(
        np.asarray(X_train), 
        y_train, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(X_train),y_train)
        )
      
      # Generate data for each
      scores = model.evaluate(np.asarray(X_train), y_train, callbacks=[WandbCallback()])
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
      mae_per_fold.append(scores[1] * 100)
      loss_per_fold.append(scores[0])

      # Increase fold number
      fold_no = fold_no + 1

      # == Provide average scores ==
      print('------------------------------------------------------------------------')
      print('Score per fold')
      for i in range(0, len(mae_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {mae_per_fold[i]}%')
      print('------------------------------------------------------------------------')
      print('Average scores for all folds:')
      print(f'> MAE: {np.mean(mae_per_fold)} (+- {np.std(mae_per_fold)})')
      print(f'> Loss: {np.mean(loss_per_fold)}')
      print('------------------------------------------------------------------------')

      wandb.log({
        "Mean Validation MAE (all folds)": np.mean(mae_per_fold),
        "Mean Validation Loss (all folds)": np.mean(loss_per_fold) 
        })

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 10
wandb.agent(keras_sequential_sweep_1, function=train, count=count)

Create sweep with ID: ut9xhi67
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/ut9xhi67


wandb: Agent Starting Run: kjp3ph4p with config:
wandb: 	batch_size: 128
wandb: 	dropout1: 0.7960750731654042
wandb: 	epochs: 30


37/37 [==============================] - 0s 2ms/step - loss: 690490432.0000 - mae: 16435.0977
Score for fold 1: loss of 690490432.0; mae of 16435.09765625
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 690490432.0 - Accuracy: 1643509.765625%
------------------------------------------------------------------------
Average scores for all folds:
> MAE: 1643509.765625 (+- 0.0)
> Loss: 690490432.0
------------------------------------------------------------------------


Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,███▇▇▆▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,████▇▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,███▇▆▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,███▇▇▆▅▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),690490432.0
Mean Validation MAE (all folds),1643509.76562
best_epoch,29
best_val_loss,690490368.0


37/37 [==============================] - 0s 2ms/step - loss: 679412352.0000 - mae: 16559.4629
Score for fold 2: loss of 679412352.0; mae of 16559.462890625
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 690490432.0 - Accuracy: 1643509.765625%
------------------------------------------------------------------------
> Fold 2 - Loss: 679412352.0 - Accuracy: 1655946.2890625%
------------------------------------------------------------------------
Average scores for all folds:
> MAE: 1649728.02734375 (+- 6218.26171875)
> Loss: 684951392.0
------------------------------------------------------------------------


...

Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),404420245.33333
Mean Validation MAE (all folds),1181918.39193
best_epoch,95
best_val_loss,400232960.0


37/37 [==============================] - 0s 1ms/step - loss: 400413216.0000 - mae: 11599.0312
Score for fold 4: loss of 400413216.0; mae of 11599.03125
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 403183264.0 - Accuracy: 1172867.3828125%
------------------------------------------------------------------------
> Fold 2 - Loss: 399276736.0 - Accuracy: 1174201.07421875%
------------------------------------------------------------------------
> Fold 3 - Loss: 410800736.0 - Accuracy: 1198686.71875%
------------------------------------------------------------------------
> Fold 4 - Loss: 400413216.0 - Accuracy: 1159903.125%
------------------------------------------------------------------------
Average scores for all folds:
> MAE: 1176414.5751953125 (+- 14019.259310381018)
> Loss: 403418488.0
--------------------------------------------------------------------

Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▇▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),403418488.0
Mean Validation MAE (all folds),1176414.5752
best_epoch,98
best_val_loss,398041088.0


37/37 [==============================] - 0s 1ms/step - loss: 406842944.0000 - mae: 11839.6719
Score for fold 5: loss of 406842944.0; mae of 11839.671875
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 403183264.0 - Accuracy: 1172867.3828125%
------------------------------------------------------------------------
> Fold 2 - Loss: 399276736.0 - Accuracy: 1174201.07421875%
------------------------------------------------------------------------
> Fold 3 - Loss: 410800736.0 - Accuracy: 1198686.71875%
------------------------------------------------------------------------
> Fold 4 - Loss: 400413216.0 - Accuracy: 1159903.125%
------------------------------------------------------------------------
> Fold 5 - Loss: 406842944.0 - Accuracy: 1183967.1875%
------------------------------------------------------------------------
Average scores for all folds:
> MAE: 11

Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▇▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),404103379.2
Mean Validation MAE (all folds),1177925.09766
best_epoch,99
best_val_loss,406842976.0


Train the model with Hyperopt to find best parameters.
Retrain model with new best parameters.
Make predictions.

In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [20]:
# Enter best params from sweep
best_params = {
    "dropout": 0.2,
    "epochs": 100,
    "batch_size": 128
}

def make_predictions(best_params):
    
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
              
    model.fit(X_train, y_train, epochs=best_params.get("epochs"),
              batch_size=best_params.get("batch_size"), verbose = 0)
    
    preds  = model.predict(X_test, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [22]:
predictions = make_predictions(best_params)

## Visualize Model Output

In [26]:
d

{'Id': RangeIndex(start=0, stop=1459, step=1),
 'SalePrice': array([[136474.11],
        [156334.36],
        [188805.05],
        ...,
        [181121.44],
        [106913.19],
        [214941.02]], dtype=float32)}

In [29]:
d = {"Id":X_test.index,"SalePrice":predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

# Revert log transform
#submission["SalePrice"] = np.exp(submission["SalePrice"])

In [30]:
submission

,Id,SalePrice
0,0,136474.109375
1,1,156334.359375
2,2,188805.046875
3,3,195929.828125
4,4,200915.140625
...,...,...
1454,1454,76588.890625
1455,1455,63704.851562
1456,1456,181121.437500
1457,1457,106913.187500


In [31]:
submission.to_csv("submission.csv",index=False)